In [1]:
"""
Encapsulation in Python with a comprehensive example showing data hiding, access control, and property decorators.
"""

'\nEncapsulation in Python with a comprehensive example showing data hiding, access control, and property decorators.\n'

In [2]:
"""
Key Encapsulation Concepts Explained:
1. Data Hiding with Access Modifiers
Public Attributes (no underscore):

account_number - Accessible from anywhere
Intended for external use

Protected Attributes (single underscore _):

_account_type - Convention-based protection
Intended for internal use and subclasses
Can still be accessed but shouldn't be

Private Attributes (double underscore __):

__balance, __pin, __account_holder - Name mangling protection
Python changes the name to _ClassName__attribute
Intended to be completely internal

2. Property Decorators (@property)
Read-Only Properties:
python@property
def balance(self):
    return self.__balance

Provides controlled access to private data
Can include validation logic
Makes private attributes appear like public ones

Getter and Setter Properties:
python@property
def account_type(self):
    return self._account_type

@account_type.setter
def account_type(self, value):
    # Validation logic here
    self._account_type = value
3. Benefits of Encapsulation Demonstrated:
Data Validation:

PIN must be 4 digits
Account type must be from valid list
Deposit/withdrawal amounts must be positive

Security:

PIN verification for sensitive operations
Admin codes for administrative functions
Transaction limits and validation

Data Integrity:

Balance can only be modified through controlled methods
Transaction history is automatically maintained
Account status controls access to operations

Interface Consistency:

Users interact with clean, simple methods
Complex internal logic is hidden
Changes to internal implementation don't affect external usage

4. Real-World Applications:
This encapsulation pattern is used in:

Banking systems (account management)
User authentication systems
Database access layers
API design with controlled access to resources
Configuration management systems

Encapsulation ensures that the internal state of objects remains consistent and secure while 
providing a clean, controlled interface for external interaction.
"""

"\nKey Encapsulation Concepts Explained:\n1. Data Hiding with Access Modifiers\nPublic Attributes (no underscore):\n\naccount_number - Accessible from anywhere\nIntended for external use\n\nProtected Attributes (single underscore _):\n\n_account_type - Convention-based protection\nIntended for internal use and subclasses\nCan still be accessed but shouldn't be\n\nPrivate Attributes (double underscore __):\n\n__balance, __pin, __account_holder - Name mangling protection\nPython changes the name to _ClassName__attribute\nIntended to be completely internal\n\n2. Property Decorators (@property)\nRead-Only Properties:\npython@property\ndef balance(self):\n    return self.__balance\n\nProvides controlled access to private data\nCan include validation logic\nMakes private attributes appear like public ones\n\nGetter and Setter Properties:\npython@property\ndef account_type(self):\n    return self._account_type\n\n@account_type.setter\ndef account_type(self, value):\n    # Validation logic her

In [8]:
import re
from datetime import datetime

class BankAccount:
    """
    Demonstrates Encapsulation with:
    - Private attributes
    - Protected attributes
    - Getter and Setter methods
    - Property decorators
    - Data validation
    """
    
    # Class variable (public)
    bank_name = "Python National Bank"
    
    def __init__(self, account_holder, initial_balance=0, account_type="Savings"):
        # Public attribute (no underscore)
        self.account_number = self.__generate_account_number()
        
        # Protected attribute (single underscore - convention only)
        self._account_type = account_type
        
        # Private attributes (double underscore - name mangling)
        self.__account_holder = account_holder
        self.__balance = 0
        self.__pin = None
        self.__transaction_history = []
        self.__is_active = True
        
        # Use setter to validate initial deposit
        if initial_balance > 0:
            self.deposit(initial_balance)
    
    # Private method (double underscore)
    def __generate_account_number(self):
        """Generate a unique account number"""
        import random
        return f"ACC{random.randint(100000, 999999)}"
    
    # Protected method (single underscore)
    def _log_transaction(self, transaction_type, amount, description=""):
        """Log transaction for internal use"""
        transaction = {
            'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'type': transaction_type,
            'amount': amount,
            'balance_after': self.__balance,
            'description': description
        }
        self.__transaction_history.append(transaction)
    
    # Property decorator - Getter for account holder
    @property
    def account_holder(self):
        """Get account holder name"""
        return self.__account_holder
    
    # Property decorator - Getter for balance (read-only)
    @property
    def balance(self):
        """Get current balance - read only"""
        if not self.__is_active:
            return "Account is inactive"
        return self.__balance
    
    # Property decorator - Getter for account type
    @property
    def account_type(self):
        """Get account type"""
        return self._account_type
    
    # Property decorator - Setter for account type
    @account_type.setter
    def account_type(self, new_type):
        """Set account type with validation"""
        valid_types = ["Savings", "Checking", "Business"]
        if new_type not in valid_types:
            raise ValueError(f"Invalid account type. Must be one of: {valid_types}")
        
        old_type = self._account_type
        self._account_type = new_type
        self._log_transaction("ACCOUNT_UPDATE", 0, f"Account type changed from {old_type} to {new_type}")
    
    # Property decorator - Getter for PIN status
    @property
    def has_pin(self):
        """Check if PIN is set"""
        return self.__pin is not None
    
    # Method to set PIN with validation
    def set_pin(self, new_pin):
        """Set PIN with validation"""
        if not isinstance(new_pin, str) or not new_pin.isdigit() or len(new_pin) != 4:
            raise ValueError("PIN must be a 4-digit string")
        
        self.__pin = new_pin
        self._log_transaction("SECURITY", 0, "PIN updated")
        return "PIN set successfully"
    
    # Method to verify PIN
    def verify_pin(self, entered_pin):
        """Verify PIN for secure operations"""
        if self.__pin is None:
            return False
        return self.__pin == entered_pin
    
    # Public method with validation
    def deposit(self, amount):
        """Deposit money with validation"""
        if not self.__is_active:
            return "Cannot deposit. Account is inactive."
        
        if not isinstance(amount, (int, float)) or amount <= 0:
            raise ValueError("Deposit amount must be a positive number")
        
        if amount > 50000:  # Daily deposit limit
            raise ValueError("Daily deposit limit is $50,000")
        
        self.__balance += amount
        self._log_transaction("DEPOSIT", amount)
        return f"Deposited ${amount:.2f}. New balance: ${self.__balance:.2f}"
    
    # Public method with PIN verification for sensitive operations
    def withdraw(self, amount, pin=None):
        """Withdraw money with PIN verification"""
        if not self.__is_active:
            return "Cannot withdraw. Account is inactive."
        
        # PIN verification for withdrawals over $500
        if amount > 500 and not self.verify_pin(pin):
            return "PIN verification required for withdrawals over $500"
        
        if not isinstance(amount, (int, float)) or amount <= 0:
            raise ValueError("Withdrawal amount must be a positive number")
        
        if amount > self.__balance:
            return "Insufficient funds"
        
        if amount > 2000:  # Daily withdrawal limit
            return "Daily withdrawal limit is $2,000"
        
        self.__balance -= amount
        self._log_transaction("WITHDRAWAL", amount)
        return f"Withdrew ${amount:.2f}. New balance: ${self.__balance:.2f}"
    
    # Public method for money transfer
    def transfer(self, recipient_account, amount, pin):
        """Transfer money to another account with PIN verification"""
        if not self.verify_pin(pin):
            return "Invalid PIN. Transfer denied."
        
        if not self.__is_active:
            return "Cannot transfer. Account is inactive."
        
        if amount > self.__balance:
            return "Insufficient funds for transfer"
        
        if amount > 5000:  # Daily transfer limit
            return "Daily transfer limit is $5,000"
        
        self.__balance -= amount
        self._log_transaction("TRANSFER_OUT", amount, f"To: {recipient_account.account_number}")
        
        # Simulate transfer to recipient
        recipient_account._receive_transfer(amount, self.account_number)
        
        return f"Transferred ${amount:.2f} to {recipient_account.account_number}. New balance: ${self.__balance:.2f}"
    
    # Protected method for receiving transfers
    def _receive_transfer(self, amount, sender_account):
        """Receive transfer from another account"""
        if self.__is_active:
            self.__balance += amount
            self._log_transaction("TRANSFER_IN", amount, f"From: {sender_account}")
    
    # Method to get transaction history (limited access)
    def get_transaction_history(self, pin, limit=10):
        """Get transaction history with PIN verification"""
        if not self.verify_pin(pin):
            return "Invalid PIN. Access denied."
        
        recent_transactions = self.__transaction_history[-limit:]
        return recent_transactions
    
    # Method to deactivate account (administrative)
    def deactivate_account(self, admin_code):
        """Deactivate account with admin verification"""
        if admin_code != "ADMIN123":
            return "Invalid admin code"
        
        self.__is_active = False
        self._log_transaction("ADMIN", 0, "Account deactivated")
        return "Account deactivated successfully"
    
    # Property decorator - Getter for account status
    @property
    def is_active(self):
        """Check if account is active"""
        return self.__is_active
    
    # String representation (doesn't expose sensitive data)
    def __str__(self):
        status = "Active" if self.__is_active else "Inactive"
        return f"Account: {self.account_number} | Holder: {self.__account_holder} | Type: {self._account_type} | Status: {status}"
    
    # Representation for debugging (still protects sensitive data)
    def __repr__(self):
        return f"BankAccount(account_number='{self.account_number}', holder='{self.__account_holder}', type='{self._account_type}')"



In [9]:

# Demonstration of Encapsulation
def demonstrate_encapsulation():
    print("=== ENCAPSULATION DEMONSTRATION ===\n")
    
    # Create bank accounts
    print("1. Creating Bank Accounts")
    alice_account = BankAccount("Alice Johnson", 1000, "Savings")
    bob_account = BankAccount("Bob Smith", 500, "Checking")
    
    print(f"Alice: {alice_account}")
    print(f"Bob: {bob_account}")
    
    # Try to access private attributes directly (will show name mangling)
    print(f"\n2. Accessing Attributes")
    print(f"Public - Account Number: {alice_account.account_number}")
    print(f"Protected - Account Type: {alice_account._account_type}")  # Convention - shouldn't access directly
    
    # Demonstrate private attribute access attempts
    print(f"\n3. Private Attribute Access Attempts")
    try:
        # This will fail - private attribute
        print(f"Direct private access: {alice_account.__balance}")
    except AttributeError as e:
        print(f"Error accessing private attribute: {e}")
    
    # Show name mangling - this works but is discouraged
    print(f"Name mangling access: {alice_account._BankAccount__balance}")
    
    # Using property decorators (proper way)
    print(f"\n4. Using Property Decorators (Proper Encapsulation)")
    print(f"Balance via property: {alice_account.balance}")
    print(f"Account holder via property: {alice_account.account_holder}")
    print(f"Account type via property: {alice_account.account_type}")
    
    # Demonstrate setter with validation
    print(f"\n5. Setter with Validation")
    try:
        alice_account.account_type = "Premium"  # This will raise an error
    except ValueError as e:
        print(f"Validation error: {e}")
    
    # Valid account type change
    alice_account.account_type = "Business"
    print(f"Account type changed to: {alice_account.account_type}")
    
    # PIN operations
    print(f"\n6. PIN Security Features")
    print(f"Has PIN initially: {alice_account.has_pin}")
    
    # Set PIN
    try:
        alice_account.set_pin("12345")  # Invalid PIN
    except ValueError as e:
        print(f"PIN validation error: {e}")
    
    print(alice_account.set_pin("1234"))  # Valid PIN
    print(f"Has PIN now: {alice_account.has_pin}")
    
    # Banking operations with encapsulation
    print(f"\n7. Banking Operations with Validation")
    
    # Deposit
    print(alice_account.deposit(500))
    
    # Small withdrawal (no PIN required)
    print(alice_account.withdraw(200))
    
    # Large withdrawal (PIN required)
    print(alice_account.withdraw(800))  # No PIN provided
    print(alice_account.withdraw(800, "1234"))  # Correct PIN
    
    # Transfer with PIN
    bob_account.set_pin("5678")
    print(f"\n8. Transfer Operations")
    print(alice_account.transfer(bob_account, 300, "1234"))
    
    # Check balances
    print(f"\nFinal Balances:")
    print(f"Alice: ${alice_account.balance}")
    print(f"Bob: ${bob_account.balance}")
    
    # Transaction history (PIN protected)
    print(f"\n9. Transaction History (PIN Protected)")
    history = alice_account.get_transaction_history("1234", 5)
    print("Recent transactions:")
    for transaction in history:
        print(f"  {transaction['timestamp']}: {transaction['type']} ${transaction['amount']} - {transaction['description']}")
    
    # Try accessing transaction history with wrong PIN
    print(f"\nWith wrong PIN: {alice_account.get_transaction_history('0000')}")
    
    # Administrative operations
    print(f"\n10. Administrative Operations")
    print(f"Account active: {alice_account.is_active}")
    print(alice_account.deactivate_account("WRONG_CODE"))  # Wrong admin code
    print(alice_account.deactivate_account("ADMIN123"))     # Correct admin code
    print(f"Account active after deactivation: {alice_account.is_active}")
    
    # Try operations on deactivated account
    print(f"\n11. Operations on Deactivated Account")
    print(alice_account.deposit(100))
    print(f"Balance of deactivated account: {alice_account.balance}")


if __name__ == "__main__":
    demonstrate_encapsulation()

=== ENCAPSULATION DEMONSTRATION ===

1. Creating Bank Accounts
Alice: Account: ACC720313 | Holder: Alice Johnson | Type: Savings | Status: Active
Bob: Account: ACC750051 | Holder: Bob Smith | Type: Checking | Status: Active

2. Accessing Attributes
Public - Account Number: ACC720313
Protected - Account Type: Savings

3. Private Attribute Access Attempts
Error accessing private attribute: 'BankAccount' object has no attribute '__balance'
Name mangling access: 1000

4. Using Property Decorators (Proper Encapsulation)
Balance via property: 1000
Account holder via property: Alice Johnson
Account type via property: Savings

5. Setter with Validation
Validation error: Invalid account type. Must be one of: ['Savings', 'Checking', 'Business']
Account type changed to: Business

6. PIN Security Features
Has PIN initially: False
PIN validation error: PIN must be a 4-digit string
PIN set successfully
Has PIN now: True

7. Banking Operations with Validation
Deposited $500.00. New balance: $1500.00
